In [2]:
import pandas as pd
import numpy as np
from pandas_datareader.famafrench import FamaFrenchReader, get_available_datasets
from datetime import datetime
import matplotlib.pyplot as plt
from utils_new import demarket, regcov, l2est
from cross_validate import cross_validate
import yfinance as yf




data = pd.read_csv('/Users/christiangartmann/Desktop/Masters/[usa]_[all_factors]_[monthly]_[vw_cap].csv')

#Get unique values
unique_values = data.iloc[:, 1].unique() 

data["date"] = pd.to_datetime(data["date"])
data = data[data["date"] >= "1980-01-01"]
data["date"] = data["date"].dt.strftime("%Y-%m")

data
ff_dict = FamaFrenchReader('10_Industry_Portfolios',
                           start='1980-01-01', 
                           end='2023-12-31').read()

ff_dict_returns = ff_dict[1] / 100
data_short = data[['name', 'date', 'ret']]

df = pd.DataFrame(data)


df['date'] = pd.to_datetime(df['date'])

df_wide = df.pivot(index='date', columns='name', values='ret')


SPX = data = pd.read_csv('/Users/christiangartmann/Desktop/Masters/S&P 500 Historical Data.csv')

SPX['Date'] = pd.to_datetime(SPX['Date'], format='%m/%d/%Y')  # Adjust format if needed

# Set 'Date' as index and keep only 'Change %' column
SPX = SPX.set_index('Date')[['Change %']]
SPX = SPX.iloc[::-1]

SPX['Change %'] = SPX['Change %'].str.rstrip('%').astype(float) / 100

from fredapi import Fred
import pandas_datareader.data as web



# Define the time range
start_date = "1980-01-01"
end_date = "2023-12-01"

# Fetch 1-month Treasury bill rate from FRED
t_bill_monthly = web.DataReader('DGS1', 'fred', start_date, end_date)

# Resample to monthly frequency (if needed)
t_bill_monthly = t_bill_monthly.resample('M').last()


t_bill_monthly = (1 + t_bill_monthly / 100) ** (1 / 12) - 1

# Ensure the index is in datetime format
t_bill_monthly.index = pd.to_datetime(t_bill_monthly.index)

# Convert end-of-month dates to start-of-month
t_bill_monthly.index = t_bill_monthly.index.to_period("M").to_timestamp(how="start")

merged_df = pd.concat([SPX, t_bill_monthly], axis=1)
merged_df['mkt_er'] = merged_df['Change %'] - merged_df['DGS1']

mkt_er = merged_df[['mkt_er']]
mkt_er_1 = mkt_er['mkt_er']
SPX_1 = SPX.squeeze()
excess_factor_returns = demarket(df_wide, SPX_1)
excess_factor_returns = excess_factor_returns[0]

/var/folders/8h/zm197vxn4g3b992t0hgg4mtr0000gn/T/ipykernel_89303/1344320445.py:25: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  end='2023-12-31').read()
/var/folders/8h/zm197vxn4g3b992t0hgg4mtr0000gn/T/ipykernel_89303/1344320445.py:25: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  end='2023-12-31').read()
/var/folders/8h/zm197vxn4g3b992t0hgg4mtr0000gn/T/ipykernel_89303/1344320445.py:25: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  end='2023-12-31').read()
/var/folders/8h/zm197vxn4g3b992t0hgg4mtr0000gn/T/ipykernel_89303/13443204